# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -2.770788772197                   -0.53    8.0    155ms
  2   -2.772137236100       -2.87       -1.31    1.0    120ms
  3   -2.772169852427       -4.49       -2.61    1.0   99.2ms
  4   -2.772170720262       -6.06       -4.06    2.0    131ms
  5   -2.772170722909       -8.58       -4.52    2.0    122ms
  6   -2.772170723012       -9.99       -5.82    1.0    107ms
  7   -2.772170723015      -11.50       -6.40    2.0    128ms
  8   -2.772170723015      -13.83       -6.74    2.0    123ms
  9   -2.772170723015      -13.94       -7.98    1.0    940ms
 10   -2.772170723015      -14.45       -8.92    2.0    138ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -2.770827882145                   -0.53    9.0    202ms
  2   -2.772058849015       -2.91       -1.31    1.0    134ms
  3   -2

1.773558015992634

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -2.770775176658                   -0.52    9.0    202ms
  2   -2.772057204464       -2.89       -1.32    1.0   98.9ms
  3   -2.772083139816       -4.59       -2.51    1.0    116ms
  4   -2.772083364859       -6.65       -3.30    1.0    106ms
  5   -2.772083414566       -7.30       -3.78    2.0    125ms
  6   -2.772083417742       -8.50       -5.25    1.0    109ms
  7   -2.772083417810      -10.17       -5.46    2.0    129ms
  8   -2.772083417811      -12.15       -6.12    1.0    118ms
  9   -2.772083417811      -13.04       -6.73    2.0    129ms
 10   -2.772083417811      -14.31       -7.29    1.0    116ms
 11   -2.772083417811      -14.03       -8.00    1.0    116ms
Solving response problem
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.